# Face detection and recognition inference pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and recogition on an image dataset using an Inception Resnet V1 pretrained on the VGGFace2 dataset.

The following Pytorch methods are included:
* Datasets
* Dataloaders
* GPU/CPU processing

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

Default params shown for illustration, but not needed. Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [3]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Define Inception Resnet V1 module

Set classify=True for pretrained classifier. For this example, we will use the model to output embeddings/CNN features. Note that for inference, it is important to set the model to `eval` mode.

See `help(InceptionResnetV1)` for more details.

In [4]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### Define a dataset and data loader

We add the `idx_to_class` attribute to the dataset to enable easy recoding of label indices to identity names later one.

In [5]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('../data/test_images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

FileNotFoundError: [Errno 2] No such file or directory: '../data/test_images'

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and detect faces and associated detection probabilities for each. The `MTCNN` forward method returns images cropped to the detected face, if a face was detected. By default only a single detected face is returned - to have `MTCNN` return all detected faces, set `keep_all=True` when creating the MTCNN object above.

To obtain bounding boxes rather than cropped face images, you can instead call the lower-level `mtcnn.detect()` function. See `help(mtcnn.detect)` for details.

In [ ]:
# aligned = []
# names = []
# for x, y in loader:
#     print(x)
#     x_aligned, prob = mtcnn(x, return_prob=True)
#     print(x_aligned.size())
#     if x_aligned is not None:
#         print('Face detected with probability: {:8f}'.format(prob))
#         aligned.append(x_aligned)
#         names.append(dataset.idx_to_class[y])

In [ ]:
names

In [ ]:
aligned

#### Calculate image embeddings

MTCNN will return images of faces all the same size, enabling easy batch processing with the Resnet recognition module. Here, since we only have a few images, we build a single batch and perform inference on it. 

For real datasets, code should be modified to control batch sizes being passed to the Resnet, particularly if being processed on a GPU. For repeated testing, it is best to separate face detection (using MTCNN) from embedding or classification (using InceptionResnetV1), as calculation of cropped faces or bounding boxes can then be performed a single time and detected faces saved for future use.

In [ ]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

#### Print distance matrix for classes

In [ ]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

In [ ]:
from PIL import Image

x = Image.open("/Users/imad/Pictures/me.jpg")
x_aligned, prob = mtcnn(x, return_prob=True)
embeddings = resnet(torch.stack([x_aligned])).detach().cpu()[0]

In [ ]:
embeddings.size()

In [6]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

class Embedder:
    def __init__(self, ):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print('Running on device: {}'.format(device))
        self.face_detecter = MTCNN(image_size=160, margin=0, min_face_size=20,
                           thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
                           device=device
                           )
        self.face_embeder = InceptionResnetV1(pretrained='vggface2').eval().to(device)
        
    def embed_one(self, img):
        x_aligned, prob = self.face_detecter(img, return_prob=True)
        embeddings = None
        if x_aligned is not None:
            embeddings = self.face_embeder(torch.stack([x_aligned])).detach().cpu()[0]
        return embeddings
    

        
        
# all necessary imports are here
import random, time, sys
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import distance
import pickle



K_random = 512
maxsize = 500
start = time.time()
R = np.random.rand(maxsize, K_random)
R = [(row, "stub value {}".format(i)) for i, row in enumerate(R)]
print(R[:3])
finish = time.time()
print("{} rows generated in {:.2f} s".format(len(R), finish - start))


class Node:
    def __init__(self, K=None, parent=None):
        assert K is not None or parent, "Either `K` should be provided for root node, or `parent` for internal nodes"
        # Reference to parent node. Used in ANNS search
        self.parent = parent
        # depth start from 0. To compute dimension, relevant to the level, use (self.depth % self.K)
        self.depth = (parent.depth + 1) if parent else 0
        # K means number of vector dimensions
        self.K = parent.K if parent else K
        # value, which splits subspace into to parts using hyperplane: item[self.depth % self.K] == self.pivot
        # pivot is empty for any leaf node.
        self.pivot = None
        # left and right child nodes
        self.left = None
        self.right = None
        # collection of items
        self.items = None
        
    def build_kd_tree(self, items, leaf_capacity=4):
        '''Takes a list of items and arranges it in a kd-tree'''
        assert items is not None, "Please provide at least one point"
        # put all items in the node if they fit into limit
        if len(items) <= leaf_capacity:
            self.items = items
        # or else split items into 2 subnodes using median value
        else:
            self.items = None
            self.left = Node(parent=self)
            self.right = Node(parent=self)
            
            #TODO 1.A.: fill in the code to initialize internal node.
            # Be careful: there may be multiple items with the same values as pivot,
            # make sure they go to the same child.
            # Also, there may be duplicate items, and you need to deal with them
            self.pivot = None     # here you should write median value with respect to coordinate
            left = None           # those items, which are smaller than the pivot value
            right = None          # those items, which are greater than the pivot value
            
            p_sorted = sorted(items, key=lambda p: p[0][self.depth % self.K])
            # if all values of a current dimension are the same, check if elements are actually the same
            if p_sorted[0][0][self.depth % self.K] == p_sorted[len(p_sorted)-1][0][self.depth % self.K]:                
                all_same = True                
                for i in range(len(p_sorted)-1):
                    # compare vectors 
                    if not np.array_equal(p_sorted[i][0], p_sorted[i+1][0]):
                        all_same = False
                        break
                if all_same:
                    # make it a leaf node
                    self.items = items
                    self.left = None
                    self.right = None
                    return self
            
            # find median and assign its value to a pivot
            med = len(p_sorted) // 2             
            self.pivot = p_sorted[med][0][self.depth % self.K]

            # set median id to the first occurence of median value
            while med > 0 and p_sorted[med - 1][0][self.depth % self.K] == self.pivot:
                med -= 1    
                
            # move pivot to the right in case if all elements in the beginning have the same value as pivot
            if med == 0 and self.pivot != p_sorted[len(p_sorted)-1][0][self.depth % self.K]:
                med = len(p_sorted) // 2
                while p_sorted[med][0][self.depth % self.K] == self.pivot:                
                    med += 1
                self.pivot = p_sorted[med][0][self.depth % self.K]
                
            left, right = p_sorted[:med], p_sorted[med:]
            
            self.left.build_kd_tree(left)
            self.right.build_kd_tree(right)

        return self
    
    def kd_find_leaf(self, key):
        ''' returns a node where key should be stored (but can be not present)'''
        if self.pivot is None or self.items is not None: # leaf node OR empty root
            return self
        else:
            
            #TODO 1.B. This is a basic operation for travesing the tree.
            # define correct path to continue recursion
            if key[self.depth % self.K] <= self.pivot:
                return self.left.kd_find_leaf(key)
            else:
                return self.right.kd_find_leaf(key)
            
#     def kd_insert_no_split(self, item):
#         '''Naive implementation of insert into leaf node. It is not used in tests of this tutorial.'''
#         node = self.kd_find_leaf(item[0])
#         node.items.append(item)
        
    def kd_insert_with_split(self, item, leaf_capacity=4):
        '''This method recursively splits the nodes into 2 child nodes if they overflow `leaf_capacity`'''
        
        #TODO 1.C. This is very simple insertion procedure.
        # Split the node if it cannot accept one more item.
        # HINT: reuse kd_find_leaf() and build_kd_tree() methods if possible
        
        node = self.kd_find_leaf(item[0])
        node.build_kd_tree((node.items or []) + [item], leaf_capacity)
        
    def get_subtree_items(self):
        '''Returns union of all items belonging to a subtree'''
        if self.pivot is None or self.items is not None: # leaf node OR empty root
            return self.items
        else:
            return self.left.get_subtree_items() + self.right.get_subtree_items()
        
    def get_nn(self, key, knn):
        '''Return K approximate nearest neighbours for a given key'''
        node = self.kd_find_leaf(key)
        best = []
        
        #TODO 1.D. ANN search.
        # write here the code which returns `knn` 
        # approximate nearest neighbours with respect to euclidean distance
        # basically, you need to move up through the parents chain until the number of elements
        # in a parent subtree is more or equal too the expected number of nearest neighbors,
        # and then return top-k elements of this subtree sorted by euclidean distance
        # HINT: you can use [scipy.spatial.]distance.euclidean(a, c) - it is already imported
        
        while node is not None and len(best) < knn:
            best = node.get_subtree_items()
            node = node.parent
        best = sorted(best, key=lambda p: distance.euclidean(p[0], key))
        
        return best[:knn]
    
    def get_in_range(self, lower_left_bound_key, upper_right_bound_key):
        '''Runs range query. Returns all items bounded by the given corners: `lower_left_bound_key`, `upper_right_bound_key`'''
        result = []
        if self.pivot is None or self.items is not None: # internal node OR empty root
            #TODO 3.B.: This is a leaf node. Select only those items from self.item
            # which fall into a given range
            for item in self.items:
                _in = [low <= v <= up for low, v, up in zip(lower_left_bound_key, item[0], upper_right_bound_key)]
                inside = all(_in)
                if inside:
                    result.append(item)
            return result
        else:
            #TODO 3.B.: This is an internal node.
            # write recursive code to collect corresponding data from subtrees
            # compare pivot to the bounds to decide whether to consider any of its children for search

            skip_right = self.pivot > upper_right_bound_key[self.depth % self.K]
            skip_left = self.pivot < lower_left_bound_key[self.depth % self.K]
            result = [] if skip_left else self.left.get_in_range(lower_left_bound_key, upper_right_bound_key)
            result += [] if skip_right else self.right.get_in_range(lower_left_bound_key, upper_right_bound_key)
            return result
        
kdtree = Node(K=K_random).build_kd_tree(R)
kdtree.get_nn(np.random.rand(K_random),5)[0][1]

[(array([5.14386584e-01, 9.84610131e-01, 6.36684220e-01, 7.93689302e-01,
       9.49291913e-01, 2.13275643e-01, 1.79537121e-01, 3.46726817e-01,
       4.71349104e-01, 4.25030690e-01, 5.43772382e-01, 6.69926620e-01,
       8.08398399e-01, 8.76255974e-01, 9.03716940e-01, 4.74764728e-01,
       4.84535149e-01, 7.89986703e-01, 8.98611121e-01, 6.96090721e-02,
       6.78247135e-01, 7.01774191e-01, 3.72568946e-01, 7.69749499e-01,
       3.24739833e-01, 4.55388605e-01, 8.77932963e-01, 7.27416248e-01,
       3.44689840e-01, 3.24775796e-01, 6.02469600e-01, 6.25700829e-02,
       5.81196731e-01, 2.41313129e-01, 8.27416069e-01, 5.23223293e-02,
       1.19909811e-01, 1.31074700e-01, 9.60701511e-01, 8.30189664e-01,
       2.59668836e-02, 4.86221732e-01, 5.29489871e-02, 2.22289967e-01,
       3.56960999e-01, 2.48916254e-01, 6.63587382e-01, 9.18813388e-02,
       9.65040370e-02, 6.20579725e-01, 2.06040846e-02, 4.93457196e-01,
       1.88230672e-01, 5.94571092e-01, 9.30771352e-02, 3.92441780e-01,
    

'stub value 481'

In [7]:
from pprint import pprint

def collate_fn(x):
    return x[0]

import os

d = '../data/data'
sub = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
loaders_list = []
datasets_list = []
for dataset_folder in sub:
    print(dataset_folder,':')
    dataset = datasets.ImageFolder(dataset_folder)
    pprint({i:c for c, i in dataset.class_to_idx.items()})
    print()
    dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
    datasets_list.append(dataset)
    loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)
    loaders_list.append(loader)

../data/data/celebs :
{0: 'angelina_jolie',
 1: 'ben_afflek',
 2: 'bradley_cooper',
 3: 'elton_john',
 4: 'jerry_seinfeld',
 5: 'kate_siegel',
 6: 'madonna',
 7: 'mindy_kaling',
 8: 'paul_rudd',
 9: 'shea_whigham'}

../data/data/friends :
{0: 'imad', 1: 'ramzy', 2: 'siroj', 3: 'yusuf', 4: 'zuhair'}



In [8]:
embder = Embedder()
Rs = []
for loader in loaders_list:
    R = []
    for x, y in loader:
        embedding = embder.embed_one(x)
        if embedding is not None:
            R.append((embedding,y))
    Rs.append(R)

Running on device: cpu


In [9]:
trees = []
for R in Rs:
    kdtree = Node(K=K_random).build_kd_tree(R)
    trees.append(kdtree)
    print(kdtree.get_nn(np.random.rand(K_random),1)[0][1])

4
2
